In [2]:
import os
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for the dataset (you can adjust these as needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the path to the folder containing your images
data_folder = 'data/sugarcaneRA/'  # Replace with the path to your data folder

# Create a dataset from the folder structure
dataset = datasets.ImageFolder(root=data_folder, transform=transform)

# Split the dataset into a training set and a test set
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Initialize the ResNet50 model and modify the classifier (fully connected) layer for your specific task
num_classes = len(dataset.classes)
model = models.vgg16(pretrained=True)
model.fc = nn.Linear(1024, num_classes)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Create data loaders for training and test sets
batch_size = 64  # Adjust as needed
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Training the model
num_epochs = 10 # Adjust as needed
train_acc_history = []  # Track training accuracy
test_acc_history = []   # Track test accuracy

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_train = 0
    correct_train = 0

    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

    # Calculate training accuracy for the current epoch
    train_accuracy = 100 * correct_train / total_train
    train_acc_history.append(train_accuracy)

# Evaluate the model on the test set
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted_test = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted_test == labels).sum().item()

# Calculate test accuracy for the current epoch
test_accuracy = 100 * correct_test / total_test
test_acc_history.append(test_accuracy)
# Print the accuracies for the current epoch
print(f"Epoch [{epoch+1}/{num_epochs}], Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")
model1=model
# Plotting the accuracy curves
epochs = range(1, num_epochs + 1)
plt.plot(epochs, train_acc_history, label='Training Accuracy')
plt.plot(epochs, test_acc_history, label='Test Accuracy')
plt.title('Training and Test Accuracy Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.show()
# torch.save(model.state_dict(), 'path_to_resnet_model.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\arnab/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:20<00:00, 26.5MB/s] 


KeyboardInterrupt: 